## State Population and Vaccination Data

In [1]:
# Import Pandas, Config Variables, and SQLalchemy
import pandas as pd
from config import endpoint, username, password
from sqlalchemy import create_engine

In [2]:
# Read CSVs for State Data
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("../data/state_covid_vaccinations_cleaned.csv")
state_latlons_df = pd.read_csv("../data/statelatlong.csv")

In [3]:
# State Populations
state_pop_df.head()

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [4]:
# State Vaccinations
state_vacc_df.head()

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,271550,191008,37120.0,26110.0,129658,17724,60964,8334
1,Alabama,1015775,636682,20717.0,12985.0,486206,9916,149288,3045
2,Arkansas,631675,483438,20932.0,16020.0,343637,11387,136415,4520
3,American Samoa,29450,16109,52883.0,28927.0,10236,18381,5860,10523
4,Arizona,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633


In [5]:
state_latlons_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [6]:
state_latlons_df = state_latlons_df.rename(columns={"State": "Abb", "City": "state"})

In [7]:
state_latlons_df.head()

,Abb,Latitude,Longitude,state
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [8]:
# State Data Merged
state_merged_df = pd.merge(state_pop_df, state_vacc_df, how="inner", on="state")
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045
1,Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334
2,Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633
3,Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520
4,California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685


In [9]:
state_merged_df = pd.merge(state_merged_df, state_latlons_df, how="left", on="state")

In [10]:
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
0,Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045,AL,32.601011,-86.680736
1,Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334,AK,61.302501,-158.775020
2,Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633,AZ,34.168219,-111.930907
3,Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520,AR,34.751928,-92.131378
4,California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685,CA,37.271875,-119.270415


In [11]:
# Set State as the Index
state_info_df = state_merged_df.set_index("state") 
state_info_df.head()

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
state,,,,,,,,,,,,
Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045,AL,32.601011,-86.680736
Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334,AK,61.302501,-158.775020
Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633,AZ,34.168219,-111.930907
Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520,AR,34.751928,-92.131378
California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685,CA,37.271875,-119.270415


In [12]:
# Add Empty Column for New Calculation
state_info_df["percent_vaccinated"] = ""
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045,AL,32.601011,-86.680736,
Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334,AK,61.302501,-158.775020,
Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633,AZ,34.168219,-111.930907,
Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520,AR,34.751928,-92.131378,
California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685,CA,37.271875,-119.270415,
Colorado,5758736,1224575,1000597,21265.0,17375.0,686889,11928,308992,5366,CO,38.997934,-105.550567,
Connecticut,3565287,914075,711142,25638.0,19946.0,494373,13866,206920,5804,CT,41.518784,-72.757507,
Delaware,973764,189050,152387,19414.0,15649.0,117600,12077,34002,3492,DE,39.145251,-75.418921,
District of Columbia,705749,193900,126500,27474.0,17924.0,87243,12362,39060,5535,DC,38.899349,-77.014567,


In [13]:
# Calcuation for percent vaccinated by state
for index, row in state_info_df.iterrows():
    population = row["population"]
    vaccinated = row["total_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_info_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [14]:
# Check the data types
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        50 non-null     int64  
 1   total_distributed                 50 non-null     int64  
 2   total_administered                50 non-null     int64  
 3   distributed_per_100K              50 non-null     float64
 4   administered_per_100K             50 non-null     float64
 5   people_with_1plus_doses           50 non-null     int64  
 6   people_with_1plus_doses_per_100K  50 non-null     int64  
 7   people_with_2_doses               50 non-null     int64  
 8   people_with_2_doses_per_100K      50 non-null     int64  
 9   Abb                               50 non-null     object 
 10  Latitude                          50 non-null     float64
 11  Longitude                         50 non-null     float64
 12  perc

In [15]:
# Convert percent vaccinated into a float
state_info_df = state_info_df.astype({"percent_vaccinated": 'float64'})
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        50 non-null     int64  
 1   total_distributed                 50 non-null     int64  
 2   total_administered                50 non-null     int64  
 3   distributed_per_100K              50 non-null     float64
 4   administered_per_100K             50 non-null     float64
 5   people_with_1plus_doses           50 non-null     int64  
 6   people_with_1plus_doses_per_100K  50 non-null     int64  
 7   people_with_2_doses               50 non-null     int64  
 8   people_with_2_doses_per_100K      50 non-null     int64  
 9   Abb                               50 non-null     object 
 10  Latitude                          50 non-null     float64
 11  Longitude                         50 non-null     float64
 12  perc

In [16]:
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045,AL,32.601011,-86.680736,12.99
Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334,AK,61.302501,-158.775020,26.11
Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633,AZ,34.168219,-111.930907,15.86
Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520,AR,34.751928,-92.131378,16.02
California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685,CA,37.271875,-119.270415,15.72
Colorado,5758736,1224575,1000597,21265.0,17375.0,686889,11928,308992,5366,CO,38.997934,-105.550567,17.38
Connecticut,3565287,914075,711142,25638.0,19946.0,494373,13866,206920,5804,CT,41.518784,-72.757507,19.95
Delaware,973764,189050,152387,19414.0,15649.0,117600,12077,34002,3492,DE,39.145251,-75.418921,15.65
District of Columbia,705749,193900,126500,27474.0,17924.0,87243,12362,39060,5535,DC,38.899349,-77.014567,17.92


## NIH AWS Data / NYT

### Set up State Master DF

In [27]:
# Read NYT / NIH json 
nytimes_nih_covid_df = pd.read_json("../data/daily_covid_stats_by_state_nyt.json", lines=True)

In [28]:
# Get Data for Most Recent Provided Date
most_recent_date = "2021-02-16"
latest_nyt_stateData = nytimes_nih_covid_df[nytimes_nih_covid_df['date'] == f'{most_recent_date}']

In [29]:
latest_nyt_stateData.head(10)

,date,state,fips,cases,deaths
19264,2021-02-16,Alabama,1,482488,9257
19265,2021-02-16,Alaska,2,56504,276
19266,2021-02-16,Arizona,4,799944,14981
19267,2021-02-16,Arkansas,5,313525,5287
19268,2021-02-16,California,6,3498760,47446
19269,2021-02-16,Colorado,8,417688,5932
19270,2021-02-16,Connecticut,9,270822,7449
19271,2021-02-16,Delaware,10,83369,1291
19272,2021-02-16,District of Columbia,11,39131,982
19273,2021-02-16,Florida,12,1837277,29153


In [30]:
import datetime as dt

for index, row in nytimes_nih_covid_df.iterrows():
    date = row["date"].strftime('%Y-%m-%d')
    nytimes_nih_covid_df.at[index, "date"] = date

In [31]:
nytimes_nih_covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19319 entries, 0 to 19318
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    19319 non-null  object
 1   state   19319 non-null  object
 2   fips    19319 non-null  int64 
 3   cases   19319 non-null  int64 
 4   deaths  19319 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 754.8+ KB


In [32]:
# Merge with State Info DF that contains population and vaccinations
latest_data_merged = pd.merge(state_info_df, latest_nyt_stateData, on="state")
latest_data_merged.head(10)

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated,date,fips,cases,deaths
0,Alabama,4903185,1015775,636682,20717.0,12985.0,486206,9916,149288,3045,AL,32.601011,-86.680736,12.99,2021-02-16,1,482488,9257
1,Alaska,731545,271550,191008,37120.0,26110.0,129658,17724,60964,8334,AK,61.302501,-158.775020,26.11,2021-02-16,2,56504,276
2,Arizona,7278717,1396475,1154583,19186.0,15862.0,886107,12174,264416,3633,AZ,34.168219,-111.930907,15.86,2021-02-16,4,799944,14981
3,Arkansas,3017804,631675,483438,20932.0,16020.0,343637,11387,136415,4520,AR,34.751928,-92.131378,16.02,2021-02-16,5,313525,5287
4,California,39512223,8250650,6210992,20881.0,15719.0,4708607,11917,1456222,3685,CA,37.271875,-119.270415,15.72,2021-02-16,6,3498760,47446
5,Colorado,5758736,1224575,1000597,21265.0,17375.0,686889,11928,308992,5366,CO,38.997934,-105.550567,17.38,2021-02-16,8,417688,5932
6,Connecticut,3565287,914075,711142,25638.0,19946.0,494373,13866,206920,5804,CT,41.518784,-72.757507,19.95,2021-02-16,9,270822,7449
7,Delaware,973764,189050,152387,19414.0,15649.0,117600,12077,34002,3492,DE,39.145251,-75.418921,15.65,2021-02-16,10,83369,1291
8,District of Columbia,705749,193900,126500,27474.0,17924.0,87243,12362,39060,5535,DC,38.899349,-77.014567,17.92,2021-02-16,11,39131,982
9,Florida,21477737,4597575,3598996,21406.0,16757.0,2441429,11367,1134793,5284,FL,27.975728,-83.833017,16.76,2021-02-16,12,1837277,29153


In [33]:
# Create a Master Dataframe to start working with
state_master_df = latest_data_merged[['state', 'Abb', 'date', 'population', 'Latitude', 'Longitude', 'cases', 'deaths', 'total_distributed', 'total_administered', 'percent_vaccinated' ]]
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated
0,Alabama,AL,2021-02-16,4903185,32.601011,-86.680736,482488,9257,1015775,636682,12.99
1,Alaska,AK,2021-02-16,731545,61.302501,-158.775020,56504,276,271550,191008,26.11
2,Arizona,AZ,2021-02-16,7278717,34.168219,-111.930907,799944,14981,1396475,1154583,15.86
3,Arkansas,AR,2021-02-16,3017804,34.751928,-92.131378,313525,5287,631675,483438,16.02
4,California,CA,2021-02-16,39512223,37.271875,-119.270415,3498760,47446,8250650,6210992,15.72
5,Colorado,CO,2021-02-16,5758736,38.997934,-105.550567,417688,5932,1224575,1000597,17.38
6,Connecticut,CT,2021-02-16,3565287,41.518784,-72.757507,270822,7449,914075,711142,19.95
7,Delaware,DE,2021-02-16,973764,39.145251,-75.418921,83369,1291,189050,152387,15.65
8,District of Columbia,DC,2021-02-16,705749,38.899349,-77.014567,39131,982,193900,126500,17.92
9,Florida,FL,2021-02-16,21477737,27.975728,-83.833017,1837277,29153,4597575,3598996,16.76


In [34]:
# Create empty column to calculate estimated percent infected to date
state_master_df["est_percent_infected_to_date"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-16,4903185,32.601011,-86.680736,482488,9257,1015775,636682,12.99,
1,Alaska,AK,2021-02-16,731545,61.302501,-158.775020,56504,276,271550,191008,26.11,
2,Arizona,AZ,2021-02-16,7278717,34.168219,-111.930907,799944,14981,1396475,1154583,15.86,
3,Arkansas,AR,2021-02-16,3017804,34.751928,-92.131378,313525,5287,631675,483438,16.02,
4,California,CA,2021-02-16,39512223,37.271875,-119.270415,3498760,47446,8250650,6210992,15.72,
5,Colorado,CO,2021-02-16,5758736,38.997934,-105.550567,417688,5932,1224575,1000597,17.38,
6,Connecticut,CT,2021-02-16,3565287,41.518784,-72.757507,270822,7449,914075,711142,19.95,
7,Delaware,DE,2021-02-16,973764,39.145251,-75.418921,83369,1291,189050,152387,15.65,
8,District of Columbia,DC,2021-02-16,705749,38.899349,-77.014567,39131,982,193900,126500,17.92,
9,Florida,FL,2021-02-16,21477737,27.975728,-83.833017,1837277,29153,4597575,3598996,16.76,


In [35]:
# Calcuation for est percent infected by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    state_master_df.at[index, "est_percent_infected_to_date"] = percent_infected

In [36]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-16,4903185,32.601011,-86.680736,482488,9257,1015775,636682,12.99,9.86
1,Alaska,AK,2021-02-16,731545,61.302501,-158.775020,56504,276,271550,191008,26.11,7.73
2,Arizona,AZ,2021-02-16,7278717,34.168219,-111.930907,799944,14981,1396475,1154583,15.86,11.01
3,Arkansas,AR,2021-02-16,3017804,34.751928,-92.131378,313525,5287,631675,483438,16.02,10.41
4,California,CA,2021-02-16,39512223,37.271875,-119.270415,3498760,47446,8250650,6210992,15.72,8.87
5,Colorado,CO,2021-02-16,5758736,38.997934,-105.550567,417688,5932,1224575,1000597,17.38,7.26
6,Connecticut,CT,2021-02-16,3565287,41.518784,-72.757507,270822,7449,914075,711142,19.95,7.61
7,Delaware,DE,2021-02-16,973764,39.145251,-75.418921,83369,1291,189050,152387,15.65,8.57
8,District of Columbia,DC,2021-02-16,705749,38.899349,-77.014567,39131,982,193900,126500,17.92,5.55
9,Florida,FL,2021-02-16,21477737,27.975728,-83.833017,1837277,29153,4597575,3598996,16.76,8.57


In [37]:
# Add Estimated Percent Immune Column
state_master_df["est_percent_immune"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-16,4903185,32.601011,-86.680736,482488,9257,1015775,636682,12.99,9.86,
1,Alaska,AK,2021-02-16,731545,61.302501,-158.775020,56504,276,271550,191008,26.11,7.73,
2,Arizona,AZ,2021-02-16,7278717,34.168219,-111.930907,799944,14981,1396475,1154583,15.86,11.01,
3,Arkansas,AR,2021-02-16,3017804,34.751928,-92.131378,313525,5287,631675,483438,16.02,10.41,
4,California,CA,2021-02-16,39512223,37.271875,-119.270415,3498760,47446,8250650,6210992,15.72,8.87,
5,Colorado,CO,2021-02-16,5758736,38.997934,-105.550567,417688,5932,1224575,1000597,17.38,7.26,
6,Connecticut,CT,2021-02-16,3565287,41.518784,-72.757507,270822,7449,914075,711142,19.95,7.61,
7,Delaware,DE,2021-02-16,973764,39.145251,-75.418921,83369,1291,189050,152387,15.65,8.57,
8,District of Columbia,DC,2021-02-16,705749,38.899349,-77.014567,39131,982,193900,126500,17.92,5.55,
9,Florida,FL,2021-02-16,21477737,27.975728,-83.833017,1837277,29153,4597575,3598996,16.76,8.57,


In [38]:
# Calcuation for estimated immune by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    infected = row["cases"]
    deaths = row["deaths"]
    vaccinated = row["total_administered"]
    
    est_total_immune = infected + vaccinated
    est_population = population - deaths
    
    percent_immune = round((est_total_immune / est_population) * 100, 2)
    
    state_master_df.at[index, "est_percent_immune"] = percent_immune

In [39]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-16,4903185,32.601011,-86.680736,482488,9257,1015775,636682,12.99,9.86,22.87
1,Alaska,AK,2021-02-16,731545,61.302501,-158.775020,56504,276,271550,191008,26.11,7.73,33.85
2,Arizona,AZ,2021-02-16,7278717,34.168219,-111.930907,799944,14981,1396475,1154583,15.86,11.01,26.91
3,Arkansas,AR,2021-02-16,3017804,34.751928,-92.131378,313525,5287,631675,483438,16.02,10.41,26.46
4,California,CA,2021-02-16,39512223,37.271875,-119.270415,3498760,47446,8250650,6210992,15.72,8.87,24.6
5,Colorado,CO,2021-02-16,5758736,38.997934,-105.550567,417688,5932,1224575,1000597,17.38,7.26,24.65
6,Connecticut,CT,2021-02-16,3565287,41.518784,-72.757507,270822,7449,914075,711142,19.95,7.61,27.6
7,Delaware,DE,2021-02-16,973764,39.145251,-75.418921,83369,1291,189050,152387,15.65,8.57,24.24
8,District of Columbia,DC,2021-02-16,705749,38.899349,-77.014567,39131,982,193900,126500,17.92,5.55,23.5
9,Florida,FL,2021-02-16,21477737,27.975728,-83.833017,1837277,29153,4597575,3598996,16.76,8.57,25.35


### Set up Daily Cases DF

In [40]:
nyt_avg_daily_cases = nytimes_nih_covid_df
nyt_avg_daily_cases.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [41]:
nyt_avg_daily_cases = nyt_avg_daily_cases.groupby("date").sum()
nyt_avg_daily_cases.head()

,fips,cases,deaths
date,,,
2020-01-21,53,1,0
2020-01-22,53,1,0
2020-01-23,53,1,0
2020-01-24,70,2,0
2020-01-25,76,3,0


In [42]:
nyt_avg_daily_cases["daily_new_cases"] = ""

In [43]:
nyt_avg_daily_cases.head()

,fips,cases,deaths,daily_new_cases
date,,,,
2020-01-21,53,1,0,
2020-01-22,53,1,0,
2020-01-23,53,1,0,
2020-01-24,70,2,0,
2020-01-25,76,3,0,


In [44]:
previous_day = 0

In [45]:
for index, row in nyt_avg_daily_cases.iterrows():
    new_cases = row["cases"] - previous_day
    previous_day = row["cases"]
    nyt_avg_daily_cases.at[index, "daily_new_cases"] = new_cases

In [46]:
nyt_avg_daily_cases["date"] = ""

In [47]:
nyt_avg_daily_cases

,fips,cases,deaths,daily_new_cases,date
date,,,,,
2020-01-21,53,1,0,1,
2020-01-22,53,1,0,0,
2020-01-23,53,1,0,0,
2020-01-24,70,2,0,1,
2020-01-25,76,3,0,1,
...,...,...,...,...,...
2021-02-12,1762,27534539,480701,99558,
2021-02-13,1762,27619266,484074,84727,
2021-02-14,1762,27683116,485154,63850,


In [48]:
for index, row in nyt_avg_daily_cases.iterrows():
    date = index
    nyt_avg_daily_cases.at[index, "date"] = date

In [49]:
nyt_avg_daily_cases.reset_index(drop=True, inplace=True)

In [50]:
nyt_avg_daily_cases.head(10)

,fips,cases,deaths,daily_new_cases,date
0,53,1,0,1,2020-01-21
1,53,1,0,0,2020-01-22
2,53,1,0,0,2020-01-23
3,70,2,0,1,2020-01-24
4,76,3,0,1,2020-01-25
5,80,5,0,2,2020-01-26
6,80,5,0,0,2020-01-27
7,80,5,0,0,2020-01-28
8,80,5,0,0,2020-01-29
9,80,6,0,1,2020-01-30


### Set Up NYT Totals Master DF

In [51]:
nyt_master = pd.merge(nytimes_nih_covid_df, state_pop_df, how ="inner", on="state")
nyt_master.head()

,date,state,fips,cases,deaths,population
0,2020-01-21,Washington,53,1,0,7614893
1,2020-01-22,Washington,53,1,0,7614893
2,2020-01-23,Washington,53,1,0,7614893
3,2020-01-24,Washington,53,1,0,7614893
4,2020-01-25,Washington,53,1,0,7614893


In [52]:
nyt_master["est_percent_infected"] = ""
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,
1,2020-01-22,Washington,53,1,0,7614893,
2,2020-01-23,Washington,53,1,0,7614893,
3,2020-01-24,Washington,53,1,0,7614893,
4,2020-01-25,Washington,53,1,0,7614893,


In [53]:
# Calcuation for est percent infected by state
for index, row in nyt_master.iterrows():
    population = int(row["population"])
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    if infected == 0:
        percent_infected = 0
    else: 
        percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    nyt_master.at[index, "est_percent_infected"] = percent_infected

In [54]:
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,0
1,2020-01-22,Washington,53,1,0,7614893,0
2,2020-01-23,Washington,53,1,0,7614893,0
3,2020-01-24,Washington,53,1,0,7614893,0
4,2020-01-25,Washington,53,1,0,7614893,0


## JSON Exports

In [55]:
# Master DF to JSON
state_master_df.to_json("../data/state-master-data.json", orient="records")

In [56]:
# NYT Master to JSON 
nyt_master.to_json("../data/nyt-master.json", orient="records")

In [57]:
nyt_avg_daily_cases.to_json("../data/daily_new_cases.json", orient="records")

### CSV Exports

In [58]:
state_master_df.to_csv("../data/state-master-data.csv")

## Amazon RDS Updates

In [59]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [60]:
# 50 States & D.C. Data
state_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [61]:
# Daily Cases Data
nyt_avg_daily_cases.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [ ]:
# Raw NTY Data
nytimes_nih_covid_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)